In [1]:
from threeML import *
from HAWCpyLike import HAWCpyLike
import region_of_interest

%matplotlib notebook

Configuration read from /home/giacomov/.threeML/threeML_config.yml
Plotter is MatPlotlib


In [2]:
ra_crab, dec_crab = 83.633083, 22.014500

roi = region_of_interest.HealpixConeROI(5.0, ra=ra_crab, dec=dec_crab)

hawc = HAWCpyLike("HAWC", 
                  "/home/giacomov/science/hawc/data/maptree_1024.root", 
                  "/home/giacomov/science/hawc/data/response.root",
                 roi)

hawc.set_active_measurements(1, 9)

hawc.display()

Response: 
----------

Response file: /home/giacomov/science/hawc/data/response.root
Number of dec bins: 106
Number of energy/nHit planes per dec bin: 10

Map Tree: 
----------



,Bin,Nside,Scheme,Obs counts,Bkg counts,obs/bkg,Pixels in ROI,Area (deg^2)
0,0,1024,RING,2.359288e+09,2.358957e+09,1.000141,23947,78.510019
1,1,1024,RING,1.563526e+08,1.562712e+08,1.000521,23947,78.510019
2,2,1024,RING,5.008916e+07,5.004267e+07,1.000929,23947,78.510019
3,3,1024,RING,1.370908e+07,1.367816e+07,1.002261,23947,78.510019
4,4,1024,RING,1.748741e+06,1.737266e+06,1.006605,23947,78.510019
5,5,1024,RING,3.882100e+05,3.829631e+05,1.013701,23947,78.510019
6,6,1024,RING,7.008800e+04,6.834725e+04,1.025469,23947,78.510019
7,7,1024,RING,4.146500e+04,4.048585e+04,1.024185,23947,78.510019
8,8,1024,RING,1.119500e+04,1.084205e+04,1.032553,23947,78.510019
9,9,1024,RING,1.546200e+04,1.503629e+04,1.028312,23947,78.510019


This Map Tree contains 911.282 transits
Total data size: 1.91576 Mbyte

Active energy/nHit planes: 
---------------------------

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [3]:
spectrum = Log_parabola()

source = PointSource("CrabNebula", ra=ra_crab, dec=dec_crab, spectral_shape=spectrum)

# NOTE: if you use units, you have to set up the values for the parameters
# AFTER you create the source, because during creation the function Log_parabola
# gets its units

spectrum.piv = 10 * u.TeV  # Pivot energy
spectrum.piv.fix = True

spectrum.K = 1e-14 / (u.TeV * u.cm**2 * u.s)  # norm (in 1/(keV cm2 s))
spectrum.K.bounds = (1e-25, 1e-19) # without units energies are in keV

spectrum.beta = 0  # log parabolic beta
spectrum.beta.bounds = (-4., 2.)

spectrum.alpha = -2.5  # log parabolic alpha (index)
spectrum.alpha.bounds = (-4., 2.)

model = Model(source)

data = DataList(hawc)

jl = JointLikelihood(model, data)

Using dec bin 57 for source CrabNebula


Widget Javascript not detected.  It may not be installed or enabled properly.


In [4]:
hawc._convolved_point_sources.size

<Quantity 1.91576 Mbyte>

In [5]:
hawc.get_log_like()

#-310426.9997955779

-665379.26889922551

In [6]:
jl.set_minimizer("ROOT")
_ = jl.fit()

#3.15e-11,2.37,42.3

Best fit values:




WARNING UserWarning: get_number_of_data_points not implemented, values for statistical measurements such as AIC or BIC are unreliable



,result,unit
parameter,,
CrabNebula.spectrum.main.Log_parabola.K,(9.88 -0.17 +0.18) x 10^-23,1 / (cm2 keV s)
CrabNebula.spectrum.main.Log_parabola.alpha,-2.790 +/- 0.020,
CrabNebula.spectrum.main.Log_parabola.beta,(1.570 +/- 0.12) x 10^-1,



Correlation matrix:



1.00,-0.17,0.66
-0.17,1.00,-0.78
0.66,-0.78,1.00



Values of -log(likelihood) at the minimum:



,-log(likelihood)
HAWC,658296.890516
total,658296.890516



Values of statistical measures:



,statistical measures
AIC,1.316592e+06
BIC,1.316594e+06


In [7]:
print spectrum(1.0 * u.TeV).to(1/(u.TeV * u.cm**2 * u.s))

2.65251009992e-11 1 / (cm2 s TeV)
